In [ ]:
import numpy as np

# Understanding PAH Property Estimation

This document provides an overview of the Python functions used to estimate the physical and chemical properties of polycyclic aromatic hydrocarbons (PAHs), specifically cyanonaphthalene. The code avoids using vibrational frequency data from the AmesPAHdb database and focuses on analytical estimates derived from Chapter 6 of The Physics and Chemistry of the Interstellar Medium by A. G. G. M. Tielens

---

## Function Reference

### `approx_surface_area_pah(Nc)`

Estimates the surface area of a PAH molecule.

- **Parameters**:  
  `Nc` (*int*): Number of carbon atoms in the molecule.  
- **Returns**:  
  `float` – Surface area in cm².

---

### `approx_radius_pah(Nc)`

Estimates the effective radius of a PAH molecule.

- **Parameters**:  
  `Nc` (*int*): Number of carbon atoms.  
- **Returns**:  
  `float` – Radius in cm.

---

### `FUV_absorption_cross_section(Nc)`

Estimates the far-ultraviolet (FUV) absorption cross section of a PAH.

- **Parameters**:  
  `Nc` (*int*): Number of carbon atoms.  
- **Returns**:  
  `float` – FUV cross section in cm².

---

### `UV_absorption_timescale(Nc, G0)`

Estimates the timescale for UV absorption by a PAH in the interstellar medium.

- **Parameters**:  
  `Nc` (*int*): Number of carbon atoms.  
  `G0` (*float*): Habing field scaling factor (UV field strength in ISM).  
- **Returns**:  
  `float` – Absorption timescale in seconds.

---

### `vibrational_degrees_of_freedom(Na)`

Computes the number of vibrational degrees of freedom in a non-linear PAH molecule.

- **Parameters**:  
  `Na` (*int*): Total number of atoms in the molecule.  
- **Returns**:  
  `int` – Number of vibrational modes.

---

### `ionization_potential(Z, Nc)`

Estimates the ionization potential of a PAH molecule.

- **Context**:  
  PAHs with ionization potentials above 13.6 eV cannot be ionized by typical interstellar UV photons due to the hydrogen absorption cutoff.

- **Parameters**:  
  `Z` (*int*): Charge of the molecule.  
  `Nc` (*int*): Number of carbon atoms.  
- **Returns**:  
  `float` – Ionization potential in eV.

---

### `photo_el_ionization_rate(Nc, G0, fy)`

Estimates the rate of photo-electron ionization rate for a PAH.

- **Parameters**:  
  `Nc` (*int*): Number of carbon atoms.  
  `G0` (*float*): Habing field.  
  `fy` (*float*): Photo-electron yield enhancement factor.  
- **Returns**:  
  `float` – Ionization rate in electrons per second.

> **Note:** This function relies on a global variable `lab_IP` (laboratory ionization potential of 1-cyanonaphthalene from the NIST database). For better accuracy, it is recommended to pass `lab_IP` as an argument.

---

### `neutral_fraction(Nc, G0, T_ISM, J_Er, J_Pe, ne)`

- **Parameters**:  
  `Nc` (*int*): Number of carbon atoms.  
  `G0` (*float*): Habing field.  
  `T_ISM` (*float*): The temperature of the diffuse interstellar medium
  `J_Pe` (*float*): Photo-electron ionization rate
  `J_Er` (*float*): Electron recombination rate
  `ne` (*int*): Electron density of given astrophysical region
- **Returns**:  
  `float` – Fraction of neutral to ionized PAH molecules

---

### Constants Used for Cyanonaphthalene

These parameter values are used for test calculations with the molecule *1-cyanonaphthalene*:

| Parameter | Description                                        | Value       |
|----------:|----------------------------------------------------|-------------|
| `Na`      | Total number of atoms                              | 19          |
| `Nc`      | Number of carbon atoms                             | 11          |
| `G0`      | Habing field (diffuse ISM)                         | 1.7         |
| `lab_IP`  | Laboratory ionization potential of cyanonaphthalene | 8.6 eV      |
| `fy`      | Yield enhancement factor for small PAHs            | 10          |
| `J_er`    | Electron recombination rate coefficient at 300K            | 1.5 × 10⁻7 cm³ s⁻¹ |
| `T_ISM`    | Temperature estimate of diffuse ISM           | 80 K |
| `ne`    | Electron density of diffuse ISM          | 1.5 × 10⁻4 cm³ |

In [51]:
# PAH FUNCTIONS
def approx_surface_area_pah(Nc): 
    s_area = 5 * 10**(-16) * Nc
    return s_area

def approx_radius_pah(Nc):
    a = 0.9 * 10**(-8) * np.sqrt(Nc)
    return a

def FUV_absorption_cross_section(Nc):
    sigma = 7 * 10**(-18) * Nc
    return sigma

def UV_absorption_timescale(Nc,G0):
    t_UV = 1.4 * 10**(9) / (Nc*G0)
    return t_UV

def vibrational_degrees_of_freedom(Na):
    return 3*Na - 6

def ionization_potential(Z, Nc):
    IP = 4.4 + (Z + 0.5) * 25.1/np.sqrt(Nc)
    return IP

def photo_el_ionization_rate(Nc, G0, fy):
    J_pe = 2.5 * 10**(-13) * (13.6-lab_IP)**2 * Nc * G0 * fy
    return J_pe

def neutral_fraction(Nc, G0, T_ISM, J_Er, J_Pe, ne):
    y0 = (J_Pe / J_Er) * np.sqrt(Nc) * G0 * np.sqrt(T_ISM) / ne
    # y1 = 3.5 * 10**(-6) * np.sqrt(Nc) * G0 * np.sqrt(T_ISM) / ne
    # y2 = 1.3 * 10**(-4) * np.sqrt(Nc) * G0 * np.sqrt(T_ISM) / ne
    y4 = (J_Pe / J_Er * ne) 
    #factor = (J_Pe / J_Er) 
    f0 = 1/(1 + y4)
    return f0

# PARAMETERS FOR CYANONAPHTHALENE
Na = 19                
Nc = 11               
G0 = 1.7              
lab_IP = 8.6          
fy = 10              
J_Er = 1.5 * 10**(-7)
T_ISM = 80
ne = 7.5 * 10**(-3)

# CALCULATIONS
print("Vibrational degrees of freedom:", vibrational_degrees_of_freedom(Na))
print("Surface area:", approx_surface_area_pah(Nc), "cm^2")
print("Radius:", approx_radius_pah(Nc), "cm")
print("FUV cross section:", FUV_absorption_cross_section(Nc), "cm^2")

t_UV = UV_absorption_timescale(Nc, G0)
print("UV absorption timescale:", t_UV, "seconds", "which is equivalent to ", t_UV / 3.154e7, "years")

print("Ionization potential (Z=0):", ionization_potential(0, Nc), "eV", "and the laboratory value for 1-cyanonaphthalene has been measured to be 8.59 - 8.61 eV")
print("Ionization potential (Z=1):", ionization_potential(1, Nc), "eV")

print("Photoelectron ionization rate:", photo_el_ionization_rate(Nc, G0, fy), "electrons s^-1")
print("Neutral fraction:", neutral_fraction(Nc, G0, T_ISM, J_Er, photo_el_ionization_rate(Nc, G0, fy), ne), "which is the ratio of neutral to ionized PAHs")

Vibrational degrees of freedom: 51
Surface area: 5.5e-15 cm^2
Radius: 2.98496231131986e-08 cm
FUV cross section: 7.7e-17 cm^2
UV absorption timescale: 74866310.16042781 seconds which is equivalent to  2.3736940444016428 years
Ionization potential (Z=0): 8.183967374450933 eV and the laboratory value for 1-cyanonaphthalene has been measured to be 8.59 - 8.61 eV
Ionization potential (Z=1): 15.751902123352803 eV
Photoelectron ionization rate: 1.1687500000000001e-09 electrons s^-1
Neutral fraction: 0.9999415659147419 which is the ratio of neutral to ionized PAHs


### Calculating the unimolecular dissociation rate

In the case of cyanonapthalene, the most likely bond to break in the molecule would be the -CN bond as given by the reaction here:
C10H7CN+ → C10H6+ + HCN + ϵ (where ϵ is the kinetic energy)